In [595]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, accuracy_score, classification_report

In [596]:
import os
os.path.abspath(os.getcwd())

'C:\\Users\\Furkan\\Downloads'

In [597]:
df = pd.read_csv("churn.csv")
df["gender"]= df["gender"].map({"Female": 0, "Male": 1}) #CINSIYETLERI 1 VE 0 DIYE SIRALADIM. ERKEK:1 KADIN:0
df.columns = df.columns.str.strip()
df.dtypes

customerID           object
gender                int64
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [598]:
df["TotalCharges"] = pd.to_numeric(df.TotalCharges, errors='coerce')
df.dtypes

customerID           object
gender                int64
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [599]:
df.drop('customerID', axis=1, inplace=True) #CUSTOMER ID GEREKSIZ OLDUGU ICIN DROPLANDI

In [600]:
df[np.isnan(df['TotalCharges'])] #TOTALCHARGES'DA BULUNAN NAN VALUELAR BELİRLENDİ

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,0,0,Yes,Yes,0,No,No phone service,DSL,Yes,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,NaN,No
753,1,0,No,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,NaN,No
936,0,0,Yes,Yes,0,Yes,No,DSL,Yes,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,NaN,No
1082,1,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,NaN,No
1340,0,0,Yes,Yes,0,No,No phone service,DSL,Yes,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,NaN,No
3331,1,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,NaN,No
3826,1,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,NaN,No
4380,0,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,NaN,No
5218,1,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,NaN,No
6670,0,0,Yes,Yes,0,Yes,Yes,DSL,No,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,NaN,No


In [601]:
mean_value=df['TotalCharges'].mean() #TOTALCHARGES'DA BULUNAN NAN VALUELARI YOK ETMEK İÇİN ORTALAMA DEĞER BULUNDU
mean_value

2283.3004408418697

In [602]:
df['TotalCharges'].fillna(value=mean_value, inplace=True) #ORTALAMA DEĞER NAN VALUELAR YERİNE YAZILDI

In [603]:
df[np.isnan(df['TotalCharges'])] #NAN VALUELARA TEKRAR BAKILDI VE HİÇ KALMADIĞI BELİRLENDİ.

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn


In [604]:

df['Churn'].replace(to_replace='Yes', value=1, inplace=True)
df['Churn'].replace(to_replace='No',  value=0, inplace=True)#CHURN HARİÇ HER ŞEY DUMMYLENDİ
df_dummies = pd.get_dummies(df)#"CHURN" COLUMN'U DUMMYLENMEYİP MAPLENDİ ÇÜNKÜ Y OLARAK ATAMAM İÇİN TEK SÜTÜNDA OLMASI GEREKİYOR.
df_dummies.head()

,gender,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,...,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,0,1,29.85,29.85,0,0,1,1,0,...,0,1,0,0,0,1,0,0,1,0
1,1,0,34,56.95,1889.50,0,1,0,1,0,...,0,0,1,0,1,0,0,0,0,1
2,1,0,2,53.85,108.15,1,1,0,1,0,...,0,1,0,0,0,1,0,0,0,1
3,1,0,45,42.30,1840.75,0,1,0,1,0,...,0,0,1,0,1,0,1,0,0,0
4,0,0,2,70.70,151.65,1,1,0,1,0,...,0,1,0,0,0,1,0,0,1,0


In [605]:
df_dummies.isnull().sum() #BAŞKA NAN VALUE VAR MI DİYE BAKILDI.

gender                                     0
SeniorCitizen                              0
tenure                                     0
MonthlyCharges                             0
TotalCharges                               0
Churn                                      0
Partner_No                                 0
Partner_Yes                                0
Dependents_No                              0
Dependents_Yes                             0
PhoneService_No                            0
PhoneService_Yes                           0
MultipleLines_No                           0
MultipleLines_No phone service             0
MultipleLines_Yes                          0
InternetService_DSL                        0
InternetService_Fiber optic                0
InternetService_No                         0
OnlineSecurity_No                          0
OnlineSecurity_No internet service         0
OnlineSecurity_Yes                         0
OnlineBackup_No                            0
OnlineBack

In [606]:
y=df_dummies.Churn.values
df2 = df_dummies
df2.drop('Churn', axis=1, inplace=True)#CHURN VALUELARI DF İÇİNDE Y VARIABLE'INA ALINDI.
df2#CHURN DIŞINDA GERİ KALAN SÜTÜNLAR DF2 İÇİNDE ALINIP X İÇİNDE VARIABLE'INA ALINDI.

,gender,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,...,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,0,1,29.85,29.85,0,1,1,0,1,...,0,1,0,0,0,1,0,0,1,0
1,1,0,34,56.95,1889.50,1,0,1,0,0,...,0,0,1,0,1,0,0,0,0,1
2,1,0,2,53.85,108.15,1,0,1,0,0,...,0,1,0,0,0,1,0,0,0,1
3,1,0,45,42.30,1840.75,1,0,1,0,1,...,0,0,1,0,1,0,1,0,0,0
4,0,0,2,70.70,151.65,1,0,1,0,0,...,0,1,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1,0,24,84.80,1990.50,0,1,0,1,0,...,1,0,1,0,0,1,0,0,0,1
7039,0,0,72,103.20,7362.90,0,1,0,1,0,...,1,0,1,0,0,1,0,1,0,0
7040,0,0,11,29.60,346.45,0,1,0,1,1,...,0,1,0,0,0,1,0,0,1,0
7041,1,1,4,74.40,306.60,0,1,1,0,0,...,0,1,0,0,0,1,0,0,0,1


In [607]:
x = df2.values
x

array([[ 0.,  0.,  1., ...,  0.,  1.,  0.],
       [ 1.,  0., 34., ...,  0.,  0.,  1.],
       [ 1.,  0.,  2., ...,  0.,  0.,  1.],
       ...,
       [ 0.,  0., 11., ...,  0.,  1.,  0.],
       [ 1.,  1.,  4., ...,  0.,  0.,  1.],
       [ 1.,  0., 66., ...,  0.,  0.,  0.]])

In [608]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.33,random_state=0)
sc = StandardScaler() #%33 TEST İÇİN AYRILDI VE GERİ KALAN %66'LIK X SC.FIT İLE EGİTİLDİ.
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [609]:
knn_model = KNeighborsClassifier(n_neighbors = 11) 
knn_model.fit(x_train,y_train) #KNN İÇİN GEREKEN KULLANILACAK KOMŞU SAYISI 11 OLARAK BELİRLENDİ.
predicted_y = knn_model.predict(x_test)
accuracy_knn = knn_model.score(x_test,y_test)#TAHMİN EDİLEN PREDICTED Y DEĞERİ X_TEST İLE GÖSTERİLDİ
print("KNN accuracy:",accuracy_knn)#ACCURACY BELİRLENMESİ İÇİN X_TEST VE Y_TEST KULLANILDI

KNN accuracy: 0.7690322580645161


In [610]:
print(classification_report(y_test, predicted_y))

              precision    recall  f1-score   support

           0       0.84      0.85      0.84      1712
           1       0.57      0.53      0.55       613

    accuracy                           0.77      2325
   macro avg       0.70      0.69      0.70      2325
weighted avg       0.76      0.77      0.77      2325



In [611]:
#LOGISTIC REGRESSION 
logr = LogisticRegression(random_state=0)
logr.fit(x_train,y_train)
y_pred = logr.predict(x_test)
confusion_matrix(y_test,y_pred)

array([[1527,  185],
       [ 276,  337]], dtype=int64)

In [612]:
accuracy_knn = logr.score(x_test,y_test)
print("Logistic Regression accuracy:",accuracy_knn)
#MODEL LOGISTIC REGRESSION ILE DE DENENDİ

Logistic Regression accuracy: 0.8017204301075269


In [613]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.85      0.89      0.87      1712
           1       0.65      0.55      0.59       613

    accuracy                           0.80      2325
   macro avg       0.75      0.72      0.73      2325
weighted avg       0.79      0.80      0.80      2325

